<a href="https://colab.research.google.com/github/iskra3138/ImageSr/blob/master/1_Image_Loading_Methods_%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image를 loading 하는 몇 가지 방법에 대한 성능 비교를 위한 노트입니다.
- GPU를 사용하지 않으므로 가속기 없이 실행해도 됩니다.

# Import Libraries

In [0]:
## Tensorflow 2.X를 사용하면 첫번째 방법인 tf.io.read_file(path)의 속도가 더 빨라집니다. (나머지 세 방법은 큰 차이 없음)
%tensorflow_version 2.x

TensorFlow 2.x selected.


성능 방법 비교를 위한 Library들을 불러오고, 각 라이브러리의 버전을 확인합니다.

In [0]:
import tensorflow as tf
print ('tensorflow vesrion: {}'.format(tf.__version__))

tensorflow vesrion: 2.1.0


In [0]:
import cv2
print ('opencv vesrion: {}'.format(cv2.__version__))

opencv vesrion: 4.1.2


In [0]:
### pillow의 경우 일반적으로 numpy를 통해 array
import PIL
from PIL import Image
import numpy as np
print ('pillow version: {}'.format(PIL.__version__))
print ('numpy vesrion: {}'.format(np.__version__))

pillow version: 6.2.2
numpy vesrion: 1.17.5


test를 위해 아래 셀을 실행하고 이미지를 다운받습니다. (Flowers DataSet)

In [0]:
#@title Download Flowers Image Data and define image path [Run Me!!!]
import pathlib
data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)

path = '/root/.keras/datasets/flower_photos/daisy/100080576_f52e8ee070_n.jpg'

228818944/228813984 [==============================] - 3s 0us/step


# 이미지 로딩 방법 비교
- 각 이미지를 로딩해서 float32형 numpy array로 만드는 데까지 걸리는 시간과 값을 측정
- 다른 모든 방법의 output은 uint8이나, Keras API를 사용하면 결과값이 float32가 되므로 각 방법에 float변환 추가

### 속도 측정

In [0]:
import time

##### 1. tensorflow - tf.io.read_file(path)
- 첫 번째 실행방법은 tf.io.read_file을 사용하는 방법입니다.
- [tf.io.read_file](https://www.tensorflow.org/api_docs/python/tf/io/read_file)의 output은 string입니다.
- 본 예제는 jpeg파일을 사용하므로, array형태로 바꿔주기 위해서 [tf.image.decode_jpeg](https://www.tensorflow.org/api_docs/python/tf/io/decode_jpeg) 를 이용합니다. 


In [0]:
times = []
for i in range(1000):
  start = time.time()
  tf_io_img = tf.io.read_file(path) ## TFRecord 파일을 만들때는 여기까지만 있어도 되므로 더 빠름
  tf_io_img = tf.image.decode_jpeg(tf_io_img).numpy()
  tf_io_img = tf_io_img.astype(np.float32)
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00151


##### 2. tensorflow - keras
- keras api를 사용해서도 이미지를 불러올 수 있습니다
  - tf.keras.preprocessing.image.load_img 의 아웃풋은 piilow Image 객체가 되고
  - tf.keras.preprocessing.image.img_to_arra의 아웃풋은 float32가 됩니다.

In [0]:
times = []
for i in range(1000):
  start = time.time()
  keras_img = tf.keras.preprocessing.image.load_img(path)
  keras_img = tf.keras.preprocessing.image.img_to_array(keras_img)
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00233


##### 3. pillow

In [0]:
times = []
for i in range(1000):
  start = time.time()
  pil_img = Image.open(path)
  pil_npimg = np.array(pil_img)
  pil_npimg = pil_npimg.astype(np.float32)
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00218


##### 4. opencv
- opencv로 이미지를 불러오면 [BGR] 순서가 되므로, 반드시 [RGB] 순서로 변경해야 합니다.

In [0]:
times = []
for i in range(1000):
  start = time.time()
  cv_img = cv2.imread(path) # BGR
  cv_img = cv2.cvtColor(cv_img,cv2.COLOR_BGR2RGB)
  cv_img = cv_img.astype(np.float32)
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00193


image를 불러와서 float32자료형태로 만드는 경우, tf.io.read_file을 쓸 때가 가장 빠르고, 그 다음이 opencv, pillow, tf.keras 순입니다.


| | tf.io.read_file | tf.keras |  pillow | opencv |
|----|----|----|----|----|
| avg.time | 0.00151 |0.00233|0.00218 | 0.00193 |
| 상대비교 | 1 | 1.54배 | 1.44 배 | 1.28배 |

### 결과값 비교

In [0]:
print (tf_io_img[0][0])
print (keras_img[0][0])
print (pil_npimg[0][0])
print (cv_img[0][0])

[133. 135. 132.]
[135. 135. 133.]
[135. 135. 133.]
[135. 135. 133.]


- **빠르기는 tf.io.read_file을 쓰는 것이 가장 빠르나, 혼자만 값이 다릅니다.**
  - 그러므로 tf.io.read_file을 사용해서 학습을 했으면, Prediction 에서도 tf.io.read_file를 사용해야 합니다!!!!

# 기타

##### 1. 이미지 크기 조정
- tf.image.resize 
```
tf.image.resize(
    images,
    size,
    method=ResizeMethod.BILINEAR,
    preserve_aspect_ratio=False,
    antialias=False,
    name=None
)
```
- The return value has the same type as images if method is ResizeMethod.NEAREST_NEIGHBOR. Otherwise, the return value has type float32. [출처](https://www.tensorflow.org/api_docs/python/tf/image/resize)

In [0]:
tf_io_img = tf.io.read_file(path)
tf_io_img = tf.image.decode_jpeg(tf_io_img)
print (tf_io_img[0][0])

tf.Tensor([133 135 132], shape=(3,), dtype=uint8)


uint8 형태의 image를 method default 옵션으로  resize

In [0]:
tf_io_img_default = tf.image.resize(tf_io_img, (224,224))
print (tf_io_img_default[0][0]) # dtype=float32 가 됨

tf.Tensor([133.64285 135.64285 132.64285], shape=(3,), dtype=float32)


uint8 형태의 image를 method 옵션을 ResizeMethod.NEAREST_NEIGHBOR으로 설정하고  resize

In [0]:
tf_io_img_NN = tf.image.resize(tf_io_img, (224,224), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
print (tf_io_img_NN[0][0]) # dtype=uint8이 됨

tf.Tensor([133 135 132], shape=(3,), dtype=uint8)


##### 2. 이미지 normalize 방법 1 
- tf.image.convert_image_dtype 
```
tf.image.convert_image_dtype(
    image,
    dtype,
    saturate=False,
    name=None
)
```
- Images that are represented using floating point values are expected to have values in the range [0,1). Image data stored in integer data types are expected to have values in the range [0,MAX], where MAX is the largest positive representable number for the data type.

[출처](https://www.tensorflow.org/api_docs/python/tf/image/convert_image_dtype)

-  tf.image.convert_image_dtype을 쓸 때, 주의해야 할 점은 input과 output의 dtype이 같으면 normalization이 작동하지 않는다.

In [0]:
# 예1) uint8 --> float32
int8_tensor = tf.constant([[[0, 100, 255]]], dtype=tf.uint8)
print (int8_tensor) ## dtype=uint8

convert_tensor = tf.image.convert_image_dtype(int8_tensor, dtype=tf.float32)
print (convert_tensor) ## dtype=float32이고 각 element는 [0,1)로 변함

tf.Tensor([[[  0 100 255]]], shape=(1, 1, 3), dtype=uint8)
tf.Tensor([[[0.        0.3921569 1.       ]]], shape=(1, 1, 3), dtype=float32)


In [0]:
# 예1) float32 --> float32
float_tensor = tf.constant([[[0.0, 100.0, 255.0]]], dtype=tf.float32)
print (float_tensor) ## dtype=float32

convert_tensor = tf.image.convert_image_dtype(float_tensor, dtype=tf.float32)
print (convert_tensor) ## dtype=float32이나, 각 element값이 하나도 안 변했음

tf.Tensor([[[  0. 100. 255.]]], shape=(1, 1, 3), dtype=float32)
tf.Tensor([[[  0. 100. 255.]]], shape=(1, 1, 3), dtype=float32)


**그러므로 tf.image.resize와 tf.image.convert_image_dtype을 함께 쓸 때는 주의해야 함**

- tf.image.resize 에 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR 을 주면 input과 dtype이 같아지므로 순서가 상관없음
- tf.image.resize 에 method 옵션이 없으면 무조건 float32 형태가 되므로 tf.image.convert_image_dtype 이 뒤에 위치하면 아무 변화 없음

In [0]:
## 예1-1) method는 없고, resize -> convert 순서
tf_io_img = tf.io.read_file(path) ## TFRecord 파일을 만들때는 여기까지만 있어도 되므로 더 빠름
tf_io_img = tf.image.decode_jpeg(tf_io_img)
print (tf_io_img[0][0]) ## dtype=uint8

tf_io_img_default = tf.image.resize(tf_io_img, (224,224))
print (tf_io_img_default[0][0]) ## dtype=float32

convert_tensor = tf.image.convert_image_dtype(tf_io_img_default, dtype=tf.float32)
print (convert_tensor[0][0]) ## dtype=float32이나, 각 element값이 하나도 안 변했음

tf.Tensor([133 135 132], shape=(3,), dtype=uint8)
tf.Tensor([133.64285 135.64285 132.64285], shape=(3,), dtype=float32)
tf.Tensor([133.64285 135.64285 132.64285], shape=(3,), dtype=float32)


In [0]:
## 예1-2) method는 없고,  convert -> resize 순서
tf_io_img = tf.io.read_file(path) ## TFRecord 파일을 만들때는 여기까지만 있어도 되므로 더 빠름
tf_io_img = tf.image.decode_jpeg(tf_io_img)
print (tf_io_img[0][0]) ## dtype=uint8

convert_tensor = tf.image.convert_image_dtype(tf_io_img, dtype=tf.float32)
print (convert_tensor[0][0]) ## dtype=float32이고, 각 element값이 [0,1)로 변했음

tf_io_img_default = tf.image.resize(convert_tensor, (224,224))
print (tf_io_img_default[0][0]) ## dtype=float32


tf.Tensor([133 135 132], shape=(3,), dtype=uint8)
tf.Tensor([0.52156866 0.5294118  0.5176471 ], shape=(3,), dtype=float32)
tf.Tensor([0.5240897  0.53193283 0.5201681 ], shape=(3,), dtype=float32)


In [0]:
## 예2-1) method가 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR이고 resize -> convert 순서
tf_io_img = tf.io.read_file(path) ## TFRecord 파일을 만들때는 여기까지만 있어도 되므로 더 빠름
tf_io_img = tf.image.decode_jpeg(tf_io_img)
print (tf_io_img[0][0]) ## dtype=uint8

tf_io_img_NN = tf.image.resize(tf_io_img, (224,224), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
print (tf_io_img_NN[0][0]) ## dtype=uint8

convert_tensor = tf.image.convert_image_dtype(tf_io_img_NN, dtype=tf.float32)
print (convert_tensor[0][0]) ## dtype=float32이고, 각 element값이 [0,1)로 변함

tf.Tensor([133 135 132], shape=(3,), dtype=uint8)
tf.Tensor([133 135 132], shape=(3,), dtype=uint8)
tf.Tensor([0.52156866 0.5294118  0.5176471 ], shape=(3,), dtype=float32)


In [0]:
## 예2-2) method가 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR이고, convert -> resize 순서
tf_io_img = tf.io.read_file(path) ## TFRecord 파일을 만들때는 여기까지만 있어도 되므로 더 빠름
tf_io_img = tf.image.decode_jpeg(tf_io_img)
print (tf_io_img[0][0]) ## dtype=uint8

convert_tensor = tf.image.convert_image_dtype(tf_io_img, dtype=tf.float32)
print (convert_tensor[0][0]) ## dtype=float32이고, 각 element값이 [0,1)로 변했음

tf_io_img_NN = tf.image.resize(convert_tensor, (224,224),  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
print (tf_io_img_NN[0][0]) ## dtype=float32 / 직전 convert_tensor의 dtype을 유지함

tf.Tensor([133 135 132], shape=(3,), dtype=uint8)
tf.Tensor([0.52156866 0.5294118  0.5176471 ], shape=(3,), dtype=float32)
tf.Tensor([0.52156866 0.5294118  0.5176471 ], shape=(3,), dtype=float32)


### 3. 이미지 normalize 방법 2 
- tf_io_img = tf.cast(img, dtype=tf.float32)/ 255.0

```
tf.cast(
    x,
    dtype,
    name=None
)
```

- tf.cast는 dtype만 바꿔 줍니다.
- 그러므로 [0,255] 사이인 image data를 [0,1]사이로 바꾸기 위해 단순하게 255.0으로 나눠주곤 합니다.
  - 참고로, tf.image.convert_image_dtype 는 [0,1) 사이로 바꿔줍니다.
- tf.cast는 input type이 무엇이든 output type으로 cast하는 것이므로 resize 와의 순서는 상관없습니다.

In [0]:
## 예1) resize -> cast 순서
tf_io_img = tf.io.read_file(path) ## TFRecord 파일을 만들때는 여기까지만 있어도 되므로 더 빠름
tf_io_img = tf.image.decode_jpeg(tf_io_img)
print (tf_io_img[0][0]) ## dtype=uint8

tf_io_img_default = tf.image.resize(tf_io_img, (224,224))
print (tf_io_img_default[0][0]) ## dtype=float32

cast_tensor = tf.cast(tf_io_img_default, dtype=tf.float32) / 255.0
print (cast_tensor[0][0]) ## dtype=float32, 각 element값이 convert_image_dtype 때와 다르게 나옴

tf.Tensor([133 135 132], shape=(3,), dtype=uint8)
tf.Tensor([133.64285 135.64285 132.64285], shape=(3,), dtype=float32)
tf.Tensor([0.52408963 0.5319328  0.52016807], shape=(3,), dtype=float32)


In [0]:
## 예2) cast -> resize 순서
tf_io_img = tf.io.read_file(path) ## TFRecord 파일을 만들때는 여기까지만 있어도 되므로 더 빠름
tf_io_img = tf.image.decode_jpeg(tf_io_img)
print (tf_io_img[0][0]) ## dtype=uint8

cast_tensor = tf.cast(tf_io_img, dtype=tf.float32) / 255.0
print (cast_tensor[0][0]) ## dtype=float32이고, 각 element값이 [0,1]로 변했음

tf_io_img_default = tf.image.resize(cast_tensor, (224,224))
print (tf_io_img_default[0][0]) ## dtype=float32


tf.Tensor([133 135 132], shape=(3,), dtype=uint8)
tf.Tensor([0.52156866 0.5294118  0.5176471 ], shape=(3,), dtype=float32)
tf.Tensor([0.5240897  0.53193283 0.5201681 ], shape=(3,), dtype=float32)


# byte string 생성 방법 비교 for making TFRecord files
- 각 이미지를 로딩해서 TFrecord 파일 입력 형태인 byte string을 만드는 데까지 걸리는 시간 및 값을 측정

### 속도 측정

In [0]:
import time

##### 1. tensorflow - tf.io.read_file(path)
- 첫 번째 실행방법은 tf.io.read_file을 사용하는 방법입니다.
- [tf.io.read_file](https://www.tensorflow.org/api_docs/python/tf/io/read_file)의 output은 string입니다.

In [0]:
times = []
for i in range(1000):
  start = time.time()
  tf_io_img_bytes = tf.io.read_file(path) ## TFRecord 파일을 만들때는 여기까지만 있어도 되므로 더 빠름
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.000217


##### 2. tensorflow - keras
- keras api를 사용해서도 이미지를 불러올 수 있습니다
  - tf.keras.preprocessing.image.load_img 의 아웃풋은 piilow Image 객체가 됩니다.

In [0]:
times = []
for i in range(1000):
  start = time.time()
  keras_img = tf.keras.preprocessing.image.load_img(path)
  keras_img = np.asarray(keras_img, np.uint8) #numpy array 변환
  keras_img_bytes = keras_img.tobytes() # bytes string 변환
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00194


##### 3. pillow

In [0]:
times = []
for i in range(1000):
  start = time.time()
  pil_img = Image.open(path)
  pil_npimg = np.asarray(pil_img)
  pil_npimg_bytes = pil_npimg.tobytes() # bytes string 변환
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00192


##### 4. opencv
- opencv로 이미지를 불러오면 [BGR] 순서가 되므로, 반드시 [RGB] 순서로 변경해야 합니다.

In [0]:
times = []
for i in range(1000):
  start = time.time()
  cv_img = cv2.imread(path) # BGR
  cv_img = cv2.cvtColor(cv_img,cv2.COLOR_BGR2RGB)
  cv_img_bytes = cv_img.tobytes() # bytes string 변환
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00167


image를 불러와서 byte string자료형태로 만드는 경우, tf.io.read_file을 쓸 때가 가장 빠르고, 그 다음이 opencv, pillow, tf.keras 순입니다.
float32를 만들때와 순서는 비슷하나, tf.io.read_file 자체가 byte_string이 output이라 다른 방법들에 비해 압도적으로 빠르다고 볼 수 있습니다.


| | tf.io.read_file | tf.keras |  pillow | opencv |
|----|----|----|----|----|
| avg.time | 0.000217 |0.00194|0.00192 | 0.00167 |
| 상대비교 | 1 | 8.94배 | 8.85 배 | 7.7배 |

### 결과값 비교

In [0]:
print (tf_io_img_bytes.numpy())

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x03\x02\x02\x03\x02\x02\x03\x03\x03\x03\x04\x03\x03\x04\x05\x08\x05\x05\x04\x04\x05\n\x07\x07\x06\x08\x0c\n\x0c\x0c\x0b\n\x0b\x0b\r\x0e\x12\x10\r\x0e\x11\x0e\x0b\x0b\x10\x16\x10\x11\x13\x14\x15\x15\x15\x0c\x0f\x17\x18\x16\x14\x18\x12\x14\x15\x14\xff\xdb\x00C\x01\x03\x04\x04\x05\x04\x05\t\x05\x05\t\x14\r\x0b\r\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\xff\xc0\x00\x11\x08\x01\x07\x01@\x03\x01\x11\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1d\x00\x00\x02\x03\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x04\x05\x02\x03\x06\x07\x01\x00\x08\t\xff\xc4\x00E\x10\x00\x02\x01\x03\x02\x04\x04\x03\x05\x05\x07\x02\x04\x07\x01\x00\x01\x02\x03\x00\x04\x11\x05!\x06\x121A\x13"Qaq\x81\x91\x07\x142B\xa1\x15#R\xb1\xd1$3br\x82\xc1\xf0C\xe1\x16S\x92\xf1\x08\x17%

In [0]:
print (keras_img_bytes)

b'\x87\x87\x85\x8a\x8a\x88\x8e\x8e\x8e\x91\x91\x91\x92\x92\x92\x93\x93\x93\x95\x95\x97\x97\x97\x99\x99\x99\x99\x97\x97\x95\x95\x95\x93\x95\x95\x93\x97\x97\x95\x99\x99\x99\x99\x98\x9d\x98\x97\x9d\x9b\x9c\xa1\x9b\x9c\xa1\x9c\x9d\xa2\x9b\x9c\xa1\x99\x9a\x9f\x97\x98\x9d\x96\x97\x9c\x96\x97\x9c\x94\x95\x9a\x95\x96\x9b\x97\x98\x9d\x98\x99\x9e\x99\x9a\x9f\x98\x99\x9d\x97\x98\x9c\x97\x98\x9c\x92\x92\x9a\x8d\x8d\x95\x87\x88\x8d\x85\x86\x8a\x87\x88\x8a\x89\x89\x89\x87\x87\x85\x84\x85\x80\x87\x87\x87\x8a\x8a\x8a\x8b\x8b\x89\x8a\x8a\x88\x89\x89\x87\x89\x8a\x85\x89\x8a\x85\x89\x8a\x85\x8a\x8b\x86\x88\x89\x84\x87\x88\x83\x89\x8a\x85\x8c\x8d\x88\x8d\x8e\x89\x8b\x8c\x87\x8a\x8b\x86\x97\x97\x95\x9d\x9d\x9d\xa0\x9f\xa4\x9c\x9d\xa2\x99\x9a\x9f\x99\x9a\x9e\x97\x99\x98\x93\x95\x92\x94\x94\x92\x92\x92\x90\x90\x90\x8e\x8e\x8e\x8c\x8b\x8c\x87\x8b\x8c\x87\x8b\x8c\x87\x8c\x8d\x88\x8c\x8d\x88\x90\x91\x8c\x92\x93\x8e\x8e\x8f\x8a\x8c\x8d\x88\x8f\x90\x8b\x92\x93\x8e\x92\x93\x8e\x91\x91\x8f\x92\x92\x92\x9c\x9c\x9e\x

In [0]:
print (pil_npimg_bytes)

b'\x87\x87\x85\x8a\x8a\x88\x8e\x8e\x8e\x91\x91\x91\x92\x92\x92\x93\x93\x93\x95\x95\x97\x97\x97\x99\x99\x99\x99\x97\x97\x95\x95\x95\x93\x95\x95\x93\x97\x97\x95\x99\x99\x99\x99\x98\x9d\x98\x97\x9d\x9b\x9c\xa1\x9b\x9c\xa1\x9c\x9d\xa2\x9b\x9c\xa1\x99\x9a\x9f\x97\x98\x9d\x96\x97\x9c\x96\x97\x9c\x94\x95\x9a\x95\x96\x9b\x97\x98\x9d\x98\x99\x9e\x99\x9a\x9f\x98\x99\x9d\x97\x98\x9c\x97\x98\x9c\x92\x92\x9a\x8d\x8d\x95\x87\x88\x8d\x85\x86\x8a\x87\x88\x8a\x89\x89\x89\x87\x87\x85\x84\x85\x80\x87\x87\x87\x8a\x8a\x8a\x8b\x8b\x89\x8a\x8a\x88\x89\x89\x87\x89\x8a\x85\x89\x8a\x85\x89\x8a\x85\x8a\x8b\x86\x88\x89\x84\x87\x88\x83\x89\x8a\x85\x8c\x8d\x88\x8d\x8e\x89\x8b\x8c\x87\x8a\x8b\x86\x97\x97\x95\x9d\x9d\x9d\xa0\x9f\xa4\x9c\x9d\xa2\x99\x9a\x9f\x99\x9a\x9e\x97\x99\x98\x93\x95\x92\x94\x94\x92\x92\x92\x90\x90\x90\x8e\x8e\x8e\x8c\x8b\x8c\x87\x8b\x8c\x87\x8b\x8c\x87\x8c\x8d\x88\x8c\x8d\x88\x90\x91\x8c\x92\x93\x8e\x8e\x8f\x8a\x8c\x8d\x88\x8f\x90\x8b\x92\x93\x8e\x92\x93\x8e\x91\x91\x8f\x92\x92\x92\x9c\x9c\x9e\x

In [0]:
print (cv_img_bytes)

b'\x87\x87\x85\x8a\x8a\x88\x8e\x8e\x8e\x91\x91\x91\x92\x92\x92\x93\x93\x93\x95\x95\x97\x97\x97\x99\x99\x99\x99\x97\x97\x95\x95\x95\x93\x95\x95\x93\x97\x97\x95\x99\x99\x99\x99\x98\x9d\x98\x97\x9d\x9b\x9c\xa1\x9b\x9c\xa1\x9c\x9d\xa2\x9b\x9c\xa1\x99\x9a\x9f\x97\x98\x9d\x96\x97\x9c\x96\x97\x9c\x94\x95\x9a\x95\x96\x9b\x97\x98\x9d\x98\x99\x9e\x99\x9a\x9f\x98\x99\x9d\x97\x98\x9c\x97\x98\x9c\x92\x92\x9a\x8d\x8d\x95\x87\x88\x8d\x85\x86\x8a\x87\x88\x8a\x89\x89\x89\x87\x87\x85\x84\x85\x80\x87\x87\x87\x8a\x8a\x8a\x8b\x8b\x89\x8a\x8a\x88\x89\x89\x87\x89\x8a\x85\x89\x8a\x85\x89\x8a\x85\x8a\x8b\x86\x88\x89\x84\x87\x88\x83\x89\x8a\x85\x8c\x8d\x88\x8d\x8e\x89\x8b\x8c\x87\x8a\x8b\x86\x97\x97\x95\x9d\x9d\x9d\xa0\x9f\xa4\x9c\x9d\xa2\x99\x9a\x9f\x99\x9a\x9e\x97\x99\x98\x93\x95\x92\x94\x94\x92\x92\x92\x90\x90\x90\x8e\x8e\x8e\x8c\x8b\x8c\x87\x8b\x8c\x87\x8b\x8c\x87\x8c\x8d\x88\x8c\x8d\x88\x90\x91\x8c\x92\x93\x8e\x8e\x8f\x8a\x8c\x8d\x88\x8f\x90\x8b\x92\x93\x8e\x92\x93\x8e\x91\x91\x8f\x92\x92\x92\x9c\x9c\x9e\x

- **빠르기는 tf.io.read_file을 쓰는 것이 가장 빠르나, 결과는 역시 혼자만 값이 다릅니다.**